Domain Understanding: Classification

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_frame=pd.read_csv('/content/Monkeypox Coursework Dataset.csv')

In [ ]:
data_frame.head()

In [ ]:
data_frame.shape

(25000, 18)

In [ ]:
data_frame.describe()

In [ ]:
data_frame.dtypes


In [ ]:
data_frame.isnull().sum()

In [ ]:
data_frame = data_frame.dropna()
data_frame

In [ ]:
data_frame.isnull().sum()

In [ ]:
data_frame.drop({'Test ID', 'Systemic Illness', 'Month of Birth', 'Health Insurance', 'Home ownership'},axis = 1)
data_frame.drop

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


class_counts = data_frame['MPOX PCR Result'].value_counts()


plt.bar(class_counts.index, class_counts.values, color=['skyblue', 'orange', 'green'])


plt.title('Distribution of Class Variable')
plt.xlabel('MPOX PCR Result')
plt.ylabel('Count')

plt.show()

data prep

In [ ]:
!pip install word2number

In [ ]:
import pandas as pd
import numpy as np
from word2number import w2n
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def prepare_data(datasave):
    # Drop rows with missing values
    df = datasave.dropna()
    print('Dropping rows with missing values')
    # Unique values in the 'Age' column
    print('Clean and transform Features')
    unique_values = df['Age'].unique()
    print("Unique values in the 'Age' Feature:")
    print(unique_values)

    # Cleaning and Transforming Age Feature and values
    def transform_age(value):
        values_to_remove = ['0', '181', '-23', '150']
        if value in values_to_remove:
            return np.nan
        else:
            try:
                return float(value)
            except ValueError:
                return w2n.word_to_num(value)

    df['Age'] = df['Age'].apply(transform_age)
    dfcleaned = df.dropna(subset=['Age'])
    print('Cleaning Age Feature and Converted to Float')
    unique_values = dfcleaned['Age'].unique()
    print("Unique values in the 'Age' Feature after cleaning:")
    print(unique_values)

    unique_values = dfcleaned['Oral Lesions'].unique()
    print("Unique values in the 'Oral Lesions' Feature:")
    print(unique_values)

    # Cleaning and transforming Oral Lesions Feature and values
    print('Cleaning and transforming Oral Lesions Feature and values')
    dfcleaned['Oral Lesions'] = dfcleaned['Oral Lesions'].replace({'YES': 1, 'No': 0})
    dfcleaned['Oral Lesions'] = dfcleaned['Oral Lesions'].astype(float)
    print("Unique values in the 'Oral Lesions' Feature after cleaning:")
    print( dfcleaned['Oral Lesions'].unique())


    # Transforming target variable into 0 and 1
    dfcleaned['MPOX PCR Result'] = dfcleaned['MPOX PCR Result'].apply(lambda x: 1 if x == 'Positive' else 0)
    print('Transforming target variable into 0 and 1')

    # Splitting into features (X) and target variable (y)

    X = dfcleaned.drop(columns=['MPOX PCR Result'], axis=1)
    y = dfcleaned['MPOX PCR Result']
    print('Splitting into features (X) and target variable (y)')

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    print('Train-Test split')
    print('X_train before normalizing')
    print(X_train)
    # Normalize count columns using MinMaxScaler
    count_columns = ['Red blood cells count', 'White blood cells count', 'Age']
    minmax_scaler = MinMaxScaler()
    print('Normalize count columns using MinMaxScaler')

    # Fit on the training data and transform both training and testing data
    X_train_normalized = X_train.copy()
    X_train_normalized[count_columns] = minmax_scaler.fit_transform(X_train[count_columns])
    print('Fit on the training data and transform both training and testing data')

    X_test_normalized = X_test.copy()
    X_test_normalized[count_columns] = minmax_scaler.transform(X_test[count_columns])
    print('Done preprocessing Data')
    return X_train_normalized, X_test_normalized,y_train,y_test

# Usage
# Assuming datasave is your original DataFrame
X_train_normalized, X_test_normalized,y_train,y_test= prepare_data(datasave)
print("Normalized X_train:")
print(X_train_normalized.head())

print(" y_train:")
print(y_train)

print("\nNormalized X_test:")
print(X_test_normalized.head())

print("y_test:")
print(y_test)

In [ ]:
X_train_normalized

Modelling


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score,precision_score,recall_score,f1_score,confusion_matrix,roc_curve

def run_classification_algorithm(algorithm, X_train, X_test, y_train, y_test):
    if algorithm == 'LogisticRegression':
        # Logistic Regression
        model = LogisticRegression()
        learnable_parameters = ['coef_', 'intercept_']
        hyperparameters = {
            'penalty': ['l1', 'l2'],
            'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'fit_intercept': [True, False],
            'solver': ['liblinear', 'saga'],
            'max_iter': [100, 200, 300, 400, 500]
        }
        source_code = "LogisticRegression()"
    elif algorithm == 'KNN':
        # K-Nearest Neighbors
        model = KNeighborsClassifier(n_neighbors=11)
        learnable_parameters = ['_tree']
        hyperparameters = {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'p': [1, 2]
        }
        source_code = "KNeighborsClassifier()"
    elif algorithm == 'DecisionTree':
        # Decision Tree
        model = DecisionTreeClassifier()
        learnable_parameters = ['tree_']
        hyperparameters = {
            'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        source_code = "DecisionTreeClassifier()"
    elif algorithm == 'SVM':
        # Support Vector Machine with RBF Kernel
        model = SVC(kernel='rbf', probability=True)
        learnable_parameters = ['dual_coef_', 'support_', 'n_support_']
        hyperparameters = {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto'],
            'kernel': ['rbf']
        }
        source_code = "SVC(kernel='rbf', probability=True)"
    elif algorithm == 'NaiveBayesGaussian':
        # Gaussian Naive Bayes
        model = GaussianNB()
        learnable_parameters = ['theta_', 'sigma_']
        hyperparameters = {}
        source_code = "GaussianNB()"
    elif algorithm == 'NaiveBayesBernoulli':
        # Bernoulli Naive Bayes
        model = BernoulliNB()
        learnable_parameters = ['class_count_', 'feature_count_']
        hyperparameters = {}
        source_code = "BernoulliNB()"
    else:
        raise ValueError("Invalid algorithm name. Choose from: LogisticRegression, KNN, DecisionTree, SVM, NaiveBayesGaussian, NaiveBayesBernoulli")

    # Train the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    fpr,tpr,_=roc_curve(y_test, y_proba) if y_proba is not None else None

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cnf_matrix = confusion_matrix(y_test, y_pred)



    # Results dictionary
    results = {
       'Modeling':{
        'Algorithm': algorithm,
        'LearnableParameters': learnable_parameters,
        'HyperParameters': hyperparameters,
        'SourceCode': source_code},
        'Evaluation':{
        'Accuracy': accuracy,
        'Precision':precision,
        'Recall':recall,
        'f1':f1,
        'confusion_matrix':cnf_matrix,
        'AUC-ROC': auc_roc,
        'fpr':fpr,
        'tpr':tpr,
        'classification_rep':classification_rep}
    }

    return results

resultsAll={}
i=0
algorithms_to_run = ['LogisticRegression', 'KNN', 'DecisionTree', 'SVM', 'NaiveBayesGaussian', 'NaiveBayesBernoulli']
for algorithm in algorithms_to_run:
    result= run_classification_algorithm(algorithm, X_train_normalized, X_test_normalized, y_train, y_test)
    resultsAll[i]=result
    i+=1

In [ ]:
for key,value in resultsAll.items():
  for k,v in value['Modeling'].items():
        print(k+':')
        print(v)
  print('------------------------------------------------------------------------------')

Evaluation


In [ ]:
for key,value in resultsAll.items():
  print(value['Modeling']['Algorithm'])
  for k,v in value['Evaluation'].items():
    if k!="confusion_matrix" and k!='fpr' and k!='tpr':
        print(k+':')
        print(v)

  %matplotlib inline
  class_names=[0,1]
  fig, ax = plt.subplots()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names)
  plt.yticks(tick_marks, class_names)
  # create heatmap
  sns.heatmap(pd.DataFrame(value['Evaluation']['confusion_matrix']), annot=True, cmap="YlGnBu" ,fmt='g')
  ax.xaxis.set_label_position("top")
  plt.tight_layout()
  plt.title('Confusion matrix for '+value['Modeling']['Algorithm'], y=1.1)
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  plt.figure(figsize=(8, 8))
  plt.plot(value['Evaluation']['fpr'], value['Evaluation']['tpr'], label=value['Modeling']['Algorithm']+ '(AUC =' + str(value['Evaluation']['AUC-ROC'])+')')
  plt.plot([0, 1], [0, 1], 'k--', label='Random')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title(f'ROC Curve - '+value['Modeling']['Algorithm'])
  plt.legend(loc='lower right')
  plt.show()
  print('-----------------------------------------------------------------------------------------')


***GridSearch best estimator SVM***



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

def grid_search_svm_rbf(X_train, y_train):
    # Define the parameter grid
    param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001]}

    # Create an SVM model with RBF kernel
    svm_model = SVC(kernel='rbf', probability=True)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(svm_model, param_grid,scoring = 'f1', cv=9, n_jobs=1, refit=True, verbose=3)

    # Perform grid search
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters

    return grid_search

grid_search = grid_search_svm_rbf(X_train_normalized, y_train)

print(grid_search)


Fitting 9 folds for each of 16 candidates, totalling 144 fits
[CV 1/9] END ....................C=0.1, gamma=1;, score=0.795 total time= 1.6min
[CV 2/9] END ....................C=0.1, gamma=1;, score=0.799 total time= 1.6min
[CV 3/9] END ....................C=0.1, gamma=1;, score=0.785 total time= 1.6min
[CV 4/9] END ....................C=0.1, gamma=1;, score=0.789 total time= 1.5min
[CV 5/9] END ....................C=0.1, gamma=1;, score=0.791 total time= 1.5min
[CV 6/9] END ....................C=0.1, gamma=1;, score=0.793 total time= 1.6min
[CV 7/9] END ....................C=0.1, gamma=1;, score=0.794 total time= 1.5min
[CV 8/9] END ....................C=0.1, gamma=1;, score=0.795 total time= 1.4min
[CV 9/9] END ....................C=0.1, gamma=1;, score=0.794 total time= 1.4min
[CV 1/9] END ..................C=0.1, gamma=0.1;, score=0.783 total time= 1.3min
[CV 2/9] END ..................C=0.1, gamma=0.1;, score=0.784 total time= 1.4min
[CV 3/9] END ..................C=0.1, gamma=0.1

In [ ]:
# Get the best model
best_svm_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_svm_model.predict(X_test_normalized)
y_proba = best_svm_model.predict_proba(X_test_normalized)[:, 1] if hasattr(best_svm_model, 'predict_proba') else None


# Evaluate the model
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None


# Print evaluation metrics
print('best hyperparameters')
print(grid_search.best_params_)
print(f"Recall: {recall:.4f}")
print(auc_roc)
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")


In [ ]:
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
class_names=[0,1]
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_pred)), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
def evaluate_svm_rbf(X_train_normalized, y_train, X_test_normalized, y_test, C=0.1, gamma=1):
    # Create SVM with RBF kernel model
    svm_rbf = SVC(kernel='rbf', probability=True, random_state=42, C=C, gamma=gamma)
    print(svm_rbf)

    # Fit the SVM model on the training data
    svm_rbf.fit(X_train_normalized, y_train)

    # Predict on the test set
    y_pred_svm_rbf = svm_rbf.predict(X_test_normalized)
    y_proba = svm_rbf.predict_proba(X_test_normalized)[:, 1] if hasattr(svm_rbf, 'predict_proba') else None

    # Evaluate the SVM with RBF kernel model
    report_svm_rbf = classification_report(y_test, y_pred_svm_rbf)
    accuracy = accuracy_score(y_test, y_pred_svm_rbf)
    auc_roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    precision = precision_score(y_test, y_pred_svm_rbf)
    recall = recall_score(y_test, y_pred_svm_rbf)
    f1 = f1_score(y_test, y_pred_svm_rbf)
    cnf_matrix = confusion_matrix(y_test, y_pred_svm_rbf)

    print("Results for SVM with RBF Kernel:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"f1: {f1:.4f}")
    print(f"auc_roc: {auc_roc:.4f}")
    print("Classification Report:")
    print(report_svm_rbf)

    # Plot confusion matrix
    class_names = [0, 1]  # name of classes
    fig, ax = plt.subplots()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)

    # create heatmap
    sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
    ax.xaxis.set_label_position("top")
    plt.tight_layout()
    plt.title('Confusion matrix', y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()
    return svm_rbf

svm_rbf=evaluate_svm_rbf(X_train_normalized, y_train, X_test_normalized, y_test, C=0.1, gamma=1)


In [ ]:
def train_evaluate_naive_bayes(X_train_normalized, y_train, X_test_normalized, y_test, model_type='bernoulli'):
    if model_type == 'gaussian':
        model = GaussianNB()
    elif model_type == 'bernoulli':
        model = BernoulliNB()
    else:
        raise ValueError("Invalid model_type. Use 'gaussian' or 'bernoulli'.")

    # Fit the Naive Bayes model on the training data
    model.fit(X_train_normalized, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test_normalized)
    y_proba = model.predict_proba(X_test_normalized)[:, 1] if hasattr(model, 'predict_proba') else None

    # Evaluate the Naive Bayes model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cnf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Results for {model_type.capitalize()} Naive Bayes:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"f1: {f1:.4f}")
    print(f"auc_roc: {auc_roc:.4f}")
    print("Classification Report:")
    print(report)

    # Plot confusion matrix
    class_names = [0, 1]
    fig, ax = plt.subplots()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)

    # create heatmap
    sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
    ax.xaxis.set_label_position("top")
    plt.tight_layout()
    plt.title(f'Confusion matrix ({model_type.capitalize()})', y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()

    return model

gnb_model = train_evaluate_naive_bayes(X_train_normalized, y_train, X_test_normalized, y_test, model_type='gaussian')
bnb_model = train_evaluate_naive_bayes(X_train_normalized, y_train, X_test_normalized, y_test, model_type='bernoulli')


**Ensemble Learning**

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

def train_evaluate_voting_classifier(base_learners, X_train_normalized, y_train, X_test_normalized, y_test):
    ensemble_learner = VotingClassifier(base_learners, voting='soft')
    ensemble_learner.fit(X_train_normalized, y_train)

    # Predict on the test set
    y_pred_ens = ensemble_learner.predict(X_test_normalized)
    y_proba = ensemble_learner.predict_proba(X_test_normalized)[:, 1] if hasattr(ensemble_learner, 'predict_proba') else None

    # Evaluate the VotingClassifier
    print('Results for Ensemble (Voting Classifier):')
    print(f"Accuracy: {accuracy_score(y_test, y_pred_ens):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred_ens):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred_ens):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred_ens):.4f}")
    auc_roc_ensemble = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    print(f"AUC-ROC: {auc_roc_ensemble:.4f}")

    # Confusion Matrix
    cnf_matrix = confusion_matrix(y_test, y_pred_ens)
    print(f"Confusion Matrix:\n{cnf_matrix}\n")

    # Plot Confusion Matrix
    %matplotlib inline
    class_names = [0, 1]  # name of classes
    fig, ax = plt.subplots()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)

    # create heatmap
    sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
    ax.xaxis.set_label_position("top")
    plt.tight_layout()
    plt.title('Confusion matrix (Ensemble)', y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()

    return ensemble_learner

ensemble_learners = [('SVM', svm_rbf), ('BernoulliNB', bnb_model)]
voting_classifier_model = train_evaluate_voting_classifier(ensemble_learners, X_train_normalized, y_train, X_test_normalized, y_test)
